In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

In [ ]:
import os
llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=os.environ.get("GROQ_API_KEY")
)


In [ ]:
from math import e


def calcular_imposto_renda(rendimento: float) -> str:
    """
    Calcula o imposto de renda com base no rendimento anual

    Args:
        rendimento (float): Rendimento anual do indivíduo.

    Returns:
        str: O valor de imposto de renda devido com base no rendimento.
    """

    if rendimento <= 2000:
        return "Você está isento de pagar imposto de renda."
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"O imposto devido é de R${imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"O imposto devido é de R${imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"O imposto devido é de R${imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"

### Convertendo função em ferramenta

In [ ]:
from llama_index.core.tools import FunctionTool

In [ ]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular imposto de renda",
    description=(
        "Calcula o imposto de renda com base no imposto anual."
        "Argumento: rendimento (float)"
        "Retorna o valor do imposto devido de acordo com faixas de rendimento"
    )
)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [ ]:
from llama_index.core.agent import AgentRunner

In [ ]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [ ]:
respose = agent_imposto.chat(
    """
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$7.500?
    """
)

In [ ]:
response = agent_imposto.chat("Quem foi Machado de Assis?")

In [ ]:
import arxiv

def consulta_artigos(titulo:str) -> str:
    """
    Consulta os artigos na base de dados ArXiv e retorna resultados formatados.
    """
    client = arxiv.Client()
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in client.results(busca)
    ]

    return "\n\n".join(resultados)

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [ ]:
agent = AgentRunner(agent_worker)
response = agent.chat("Me retorne artigos sobre LangChain na educação")

In [ ]:
### Usando o Tavily 

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [ ]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [ ]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

In [ ]:
tavily_tool.search("Me retorne artigos científicos sobre LangChain", max_results=3)

In [ ]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description="Busca artigos com Tavily sobre um determinado tópico"
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("Me retorne artigos sobre LLM e LangChain")

In [ ]:
print(response)

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [ ]:
url = "files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
url = "files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

### Gerar os Embeddings

In [ ]:
from  llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-large",
)

In [ ]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [ ]:
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

### Engine de busca

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

In [ ]:
store_context = StorageContext.from_defaults(persist_dir="artigo")
artigo_index = load_index_from_storage(store_context)

store_context = StorageContext.from_defaults(persist_dir="tutorial")
tutorial_index = load_index_from_storage(store_context)

In [ ]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similarity_top_k=3, llm=llm)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre LLM e LangChain."
                "Use uma pergunta detalhada em texto simples como entrada para ferramenta."
            )
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "Fornece informações sobre casos de uso e aplicações em LLMs."
                "Use uma pergunta detalhada em texto simples como entrada para ferramenta."
            )
        )
    )
]

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

In [ ]:
response = agent_document.chat(
    """
    Quais são os principais casos de uso de LLMs e LangChain?
    """
)

In [ ]:
response = agent_document.chat(
    """
    Quais as principais tendências em LangChain e LLMs?
    """
)

In [ ]:
response = agent_document.chat(
    """
    Quais as principais tendências em LangChain que eu deveria estudar?
    """
)

### Agente ReAct

In [ ]:
from llama_index.core.agent import ReActAgent

In [ ]:
agent = ReActAgent.from_tools(
    tools=query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [ ]:
response = agent.chat(
    """
    Quais são os principais ferramentas usadas em LangChain?
    """
)